In [1]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
          "spark.sql.catalogImplementation":"hive"
         }
}

In [2]:
print(sc.getConf().get("spark.sql.shuffle.partitions"))

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1656022220518_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

None

In [3]:
import tecton
import pandas as pd

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
dataset_location = "s3://gd-gdmltecton-stage-feature-data/wdd-model/wdd_us_labeled_20211105/full.parquet/dataset_00000_no_orders.parquet/"
wdd_df = spark.read.parquet(dataset_location)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
import pyspark.sql.functions as F
import sys

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
spine = (wdd_df
         .select('shopper_id')
         .withColumn('bill_modified_mst_ts', F.lit(pd.Timestamp('2022-05-21 12:00Z')))
        )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
spine.cache().limit(10).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+
|shopper_id|bill_modified_mst_ts|
+----------+--------------------+
|  10000543| 2022-05-21 12:00:00|
| 100013078| 2022-05-21 12:00:00|
|  10001816| 2022-05-21 12:00:00|
| 100027849| 2022-05-21 12:00:00|
| 100031509| 2022-05-21 12:00:00|
| 100031972| 2022-05-21 12:00:00|
| 100032819| 2022-05-21 12:00:00|
| 100036978| 2022-05-21 12:00:00|
| 100047907| 2022-05-21 12:00:00|
| 100051835| 2022-05-21 12:00:00|
+----------+--------------------+

In [9]:
prod = tecton.get_workspace('prod')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# This feature service is trimmed down due to our previous issues with one of the features
# It works now and is very fast
wdd_service = prod.get_feature_service("wdd_service")
features = wdd_service.get_historical_features(spine).to_spark()
features.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

54442

In [11]:
fvs = tecton.list_feature_views()
print(f"We have {len(fvs)} feature views and all of these all work and are fast individually (Jason tested these Fri 6/17) with the same 54k id spine above")
print()
print(fvs)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We have 44 feature views and all of these all work and are fast individually (Jason tested these Fri 6/17) with the same 54k id spine above

['business_registrations_orders', 'business_registrations_orders_lifetime', 'certified_domains_orders', 'certified_domains_orders_lifetime', 'cnp_hosting_orders', 'cnp_hosting_orders_lifetime', 'domain_alert_orders', 'domain_alert_orders_lifetime', 'domain_marketplace_orders', 'domain_marketplace_orders_lifetime', 'domain_name_registration_orders', 'domain_name_registration_orders_lifetime', 'domain_name_transfer_orders', 'domain_name_transfer_orders_lifetime', 'domain_ownership_protection_orders', 'domain_ownership_protection_orders_lifetime', 'domain_services_orders', 'domain_services_orders_lifetime', 'domains_by_proxy_orders', 'domains_by_proxy_orders_lifetime', 'email_orders', 'email_orders_lifetime', 'grid_orders', 'grid_orders_lifetime', 'last_order', 'ms_office_365_orders', 'ms_office_365_orders_lifetime', 'non_top_20_orders', 'non_top_20_

In [12]:
# However, this feature service with many, if not all, of those feature views in it seems to disconnect the session
# which is typical behavior for running out of memory
shopper_ml_service = tecton.get_feature_service("shopper_ml_service")
features = shopper_ml_service.get_historical_features(spine.limit(1)).to_spark()
features.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1

In [17]:
from py4j.java_gateway import java_import
java_import(spark._sc._jvm, "org.apache.spark.sql.api.python.*")

features.explain()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [shopper_id#0, bill_modified_mst_ts#141, domain_name_registration_orders.domain_name_registration_receipt_price_amt_sum_90d_1d#1126, domain_name_registration_orders.domain_name_registration_receipt_qty_sum_90d_1d#1131L, shared_hosting_orders.shared_hosting_receipt_price_amt_sum_90d_1d#1218, shared_hosting_orders.shared_hosting_receipt_qty_sum_90d_1d#1223L, domains_by_proxy_orders.domains_by_proxy_receipt_price_amt_sum_90d_1d#1316, domains_by_proxy_orders.domains_by_proxy_receipt_qty_sum_90d_1d#1321L, website_builder_orders.website_builder_receipt_price_amt_sum_90d_1d#1420, website_builder_orders.website_builder_receipt_qty_sum_90d_1d#1425L, wordpress_managed_plans_orders.wordpress_managed_plans_receipt_price_amt_sum_90d_1d#1530, wordpress_managed_plans_orders.wordpress_managed_plans_receipt_qty_sum_90d_1d#1535L, ms_office_365_orders.ms_office_365_receipt_price_amt_sum_90d_1d#1646, ms_office_365_orders.ms_office_365_rece

In [18]:
one_item = spine.limit(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
data_frames = list(map(
    lambda fv: prod.get_feature_view(fv).get_historical_features(one_item).to_spark(), 
    fvs))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
data_frames[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[shopper_id: string, bill_modified_mst_ts: timestamp, business_registrations_orders.business_registrations_receipt_price_amt_sum_90d_1d: double, business_registrations_orders.business_registrations_receipt_qty_sum_90d_1d: bigint]

In [22]:
for df in data_frames:
    df.cache()
    print(df.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[shopper_id: string, bill_modified_mst_ts: timestamp, business_registrations_orders.business_registrations_receipt_price_amt_sum_90d_1d: double, business_registrations_orders.business_registrations_receipt_qty_sum_90d_1d: bigint]
1
DataFrame[shopper_id: string, bill_modified_mst_ts: timestamp, business_registrations_orders_lifetime.business_registrations_order_count: bigint, business_registrations_orders_lifetime.business_registrations_total_quantity: bigint, business_registrations_orders_lifetime.business_registrations_total_spent: double]
1
DataFrame[shopper_id: string, bill_modified_mst_ts: timestamp, certified_domains_orders.certified_domains_receipt_price_amt_sum_90d_1d: double, certified_domains_orders.certified_domains_receipt_qty_sum_90d_1d: bigint]
1
DataFrame[shopper_id: string, bill_modified_mst_ts: timestamp, certified_domains_orders_lifetime.certified_domains_order_count: bigint, certified_domains_orders_lifetime.certified_domains_total_quantity: bigint, certified

In [26]:
from functools import reduce
from pyspark.sql import functions as F
data_frames_spark = reduce(
    lambda df1, df2: df1.join(df2, ["shopper_id"]), 
    data_frames[:1])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
data_frames_spark.explain()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
InMemoryTableScan [shopper_id#0, bill_modified_mst_ts#141, business_registrations_orders.business_registrations_receipt_price_amt_sum_90d_1d#23290, business_registrations_orders.business_registrations_receipt_qty_sum_90d_1d#23295L]
   +- InMemoryRelation [shopper_id#0, bill_modified_mst_ts#141, business_registrations_orders.business_registrations_receipt_price_amt_sum_90d_1d#23290, business_registrations_orders.business_registrations_receipt_qty_sum_90d_1d#23295L], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(17) Project [shopper_id#0, bill_modified_mst_ts#141, business_registrations_orders.business_registrations_receipt_price_amt_sum_90d_1d#23290, business_registrations_orders.business_registrations_receipt_qty_sum_90d_1d#23295L]
            +- *(17) SortMergeJoin [__tecton_temporal_join_column#23276, shopper_id#0], [__tecton_temporal_join_column#23280, shopper_id#23248], LeftOuter
               :- *(3) Sort [__tecton_temporal_join_column#232

In [17]:
data_frames_spark.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1

In [21]:
data_frames_spark.explain()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'JavaPackage' object is not callable
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 386, in explain
    print(self._sc._jvm.PythonSQLUtils.explainString(self._jdf.queryExecution(), explain_mode))
TypeError: 'JavaPackage' object is not callable



In [19]:
data_frames_spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[bill_modified_mst_ts: timestamp, business_registrations_orders.business_registrations_receipt_price_amt_sum_90d_1d: double, business_registrations_orders.business_registrations_receipt_qty_sum_90d_1d: bigint, shopper_id: string, bill_modified_mst_ts: timestamp, business_registrations_orders_lifetime.business_registrations_order_count: bigint, business_registrations_orders_lifetime.business_registrations_total_quantity: bigint, business_registrations_orders_lifetime.business_registrations_total_spent: double, bill_modified_mst_ts: timestamp, certified_domains_orders.certified_domains_receipt_price_amt_sum_90d_1d: double, certified_domains_orders.certified_domains_receipt_qty_sum_90d_1d: bigint, bill_modified_mst_ts: timestamp, certified_domains_orders_lifetime.certified_domains_order_count: bigint, certified_domains_orders_lifetime.certified_domains_total_quantity: bigint, certified_domains_orders_lifetime.certified_domains_total_spent: double, bill_modified_mst_ts: timestamp,

In [22]:
len(data_frames_spark.columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

162